In [14]:
import numpy as np
import csv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from os import listdir
        
        



# test_data = np.genfromtxt('./data_export_cobercat/data_export_1-2015.csv',skip_header=1,delimiter=',', dtype=None, names=['timestamp','date','lat','lng','signal_inst','signal_min','signal_max','signal_avg','carrier','fullCarrier','status','net','net_type','lac','cid','psc','speed,satellites','precision1','provider','activity','incident','downloadSpeed','uploadSpeed'],filling_values=(-1))

In [54]:
class cell_user:
    '''Holds data for each user'''
    
    def __init__(self, inpu):
        self.timestamp = int(inpu[0])
        self.date = inpu[1]
        self.lat = float(inpu[2])
        self.lng = float(inpu[3])
        self.signal_inst = int(inpu[4])
        self.signal_min = int(inpu[5])
        self.signal_max = int(inpu[6])
        self.signal_avg = int(inpu[7])
        self.carrier = inpu[8]
        self.fullCarrier = inpu[9]
        self.status = int(inpu[10])
            
#         self.status = int(inpu[10])
        self.net = int(inpu[11])
        self.net_type = inpu[12]
        self.lac = int(inpu[13])
        self.cid = int(inpu[14])
        self.psc = int(inpu[15])
        self.speed = float(inpu[16])
        self.satellites = int(inpu[17])
        self.precision1 = int(inpu[18])
        self.provider = inpu[19]
        self.activity = int(inpu[20])
        try:
            self.incident = inpu[21]
            if len(self.incident) == 0:
                self.incident = -1
            
        except:
            self.incident = -1
            
        try:
            self.downloadSpeed = inpu[22]
            self.uploadSpeed = inpu[23]
            
        except:
            self.downloadSpeed = -1
            self.uploadSpeed = -1
        
        
        
    def __str__(self):
        '''return the name upon str(object)'''
        return str(self.timestamp)


In [55]:
unix = "./data_export_cobercat/data_export_1-2015.csv"

phone_data = open(unix, "r")
phone_data_reader = csv.reader(phone_data, delimiter=',')
phone_data_list = list(phone_data_reader)
list_of_phone_data = []


for x in phone_data_list[1:]:
    try:
        y = cell_user(x)
        list_of_phone_data.append (y)
    except:
        print x[0]
        
        
print list_of_phone_data[-1]

1449354846000
1449345888247
1449344711000
1449341477778
1449340195644
1449339283000
1449338227000
1449337323000
1449336052193
1449335122000
1449335061086
1449334995596
1449334939072
1449334878066
1449334817059
1449334756065
1449334697044
1449334636046
1449334575032
1449334455000
1449334393000
1449334333008
1449334273000
1449334213000
1449334150988
1449334092000
1449334035000
1449333970000
1449333910000
1449333850000
1449333787949
1449333728000
1449333667935
1449332909853
1449332790000
1449332727834
1449332667000
1449332604000
1449332361000
1449331566705
1449331505712
1449330899000
1449330774623
1449330630000
1449330470591
1449329866533
1449329346000
1449329287000
1449329224000
1449329101000
1449328666408
1449328421000
1449328360000
1449327997000
1449321534614
1449321362000
1449321232000
1449321051000
1449320901695
1449320761000
1449320510508
1449320086000
1449319843000
1449319672418
1449318702310
1449318595000
1449318403394
1449317798000
1449315987016
1449315861000
1449315737982
144931

In [51]:
for x in list_of_phone_data[-10:-5]:
    print x.uploadSpeed

In [52]:
sum = 0
count = 0
for x in list_of_phone_data:
    sum += float(getattr(x, "speed"))
    count += 1

In [53]:
print sum/count

26.1284117812
